# Transformée en Ondelettes 2D, application au traitement des images

**BOULKAID, Yasmine**


**IVELAND, Edda**
  
**4GMM TP de Signal**

Introduction: 
Le but de cet compte rendu de TP est de . 

In [ ]:
#import des libraries utilisées dans ce TP
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests
import boto3
import os

In [ ]:
s3_endpoint_url = 'https://object-rook-ceph.apps.math.cnrs.fr/'
s3_access_key_id = '9F7EB8YBUWXDV7A4IZYW' # le contenu de secrets/dossal
s3_secret_access_key = 'skV01Eei5M3xVOxROIDr3qymYhWtkrxPpMyj8nwb' # le contenu de secrets/dossal
s3_bucket = 'signal-image'
s3 = boto3.client('s3',
                  '',
                  endpoint_url = s3_endpoint_url,
                  aws_access_key_id = s3_access_key_id,
                  aws_secret_access_key = s3_secret_access_key)
Data=["Lenna.jpg","Canaletto.jpeg","MinotaureBruite.jpeg","Cartoon.jpg"]
if not os.path.isfile('Lenna.jpg'):
    for filenames in Data:  
        s3.download_file(s3_bucket,filenames,filenames)
def chargeData(name):
    if name=='Lenna':
        res=np.array(Image.open("Lenna.jpg")).astype(float)
    if name=='Canaletto':
        res=np.array(Image.open("Canaletto.jpeg")).astype(float)
    if name=='Minotaure':
        res=np.array(Image.open("MinotaureBruite.jpeg")).astype(float)  
    if name=='Cartoon':
        res=np.array(Image.open("Cartoon.jpg")).astype(float) 
    return res
options1=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None)
options2=dict(width=700,height=400,xaxis=None,yaxis=None,toolbar=None)

## Approximation linéaire et non linéaire.

In [ ]:
im2=chargeData('Lenna')
im=chargeData('Canaletto')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options),hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
size=400
WT= pywt.wavedecn(im, 'haar', mode='per', level=2)
arr, coeff_slices = pywt.coeffs_to_array(WT)
hv.Image(arr).opts(cmap='gray',width=size,height=size)

Ecrire une fonction qui réalise une approxiamtion non linéaire en seuillant les coefficients d'ondelettes.
On pourra utiliser les fonctions suivante : pywt.coeffs_to_array et pywt.array_to_coeffs

In [ ]:
def ApproxOnd2D(I,qmf,L,threshold):
    Lmax=pywt.dwtn_max_level(np.shape(I),pywt.Wavelet(qmf))
    L1=min(L,Lmax)
    WTB= pywt.wavedecn(I, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WTB)
    WTS=arr*(np.abs(arr)>threshold)
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices)
    Irec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    ncoeffs = len(coeffs_from_arr)
    return Irec,ncoeffs


**Tester la fonction d'approximation 2D :**

In [ ]:
Irec,ncoeffs=ApproxOnd2D(im,'haar',9,50)
pn.Row(hv.Image(im,label="Image originale").opts(**options),hv.Image(Irec,label="Approximation non linéaire avec seuillage").opts(**options))

**Nous remarquons que l'image de base est la plus claire. En testant differentes valeurs de L et du seuil, nous voyons qu'en augumentant les deux valeurs, l'approximation est de moins bonne qualité. En effet, la fonction "ApproxOnd2D" ne garde que les coefficients d'ondelettes qui sont plus grands que le seuil. Alors quand le seuil augumente, il va y avoir moins de coefficients restants pour reconstruir l'image et donc l'image sera plus floue.**

**En testant differentes bases d'ondelettes, on observe l'impact sur la résolution d'image reconstruite. Les ondelettes de Haar, étant constantes par morceaux, entraînent l'apparition de rectangles dans l'image reconstruite, contrairement aux ondelettes de Daubechies.**

Ecrire une focntion PSNR.

In [ ]:
def PSNR(I,Iref):
    mse = np.mean( (Iref - I) ** 2 )
    if mse == 0:
        return 100
    Val_MAX = np.max(Iref)
    return 20 * np.log10(Val_MAX / np.sqrt(mse))
    

In [ ]:
#Test de la fonction PSNR
PSNR(Irec,im) #pour la sortie du test du fonction ApproxOnd2D

**Fonction qui calcule le Peak Signal to Noise Ratio (PSNR), une mesure de distortion en image numérique qui permet de quantifier la qualité de reconstruction de l'image compressée par rapport à l'image originale. En général,plus le PSNR est élevé meilleure est la qualité de l'image recontruite.**

Ecrire une fonction qui réalise une approximation non linéaire en conservant un nombre N de coefficients d'ondelettes et la tester. On pourra utiliser les fonctions pywt.ravel_coeffs et unravel_coeffs.

In [ ]:
def ApproxOnd2nonlin(I,qmf,L,N): #Retourne Irec et p le PSNR
    N1=np.shape(I)[0]
    N2=np.shape(I)[1]
    Lmax=pywt.dwtn_max_level(np.shape(I),pywt.Wavelet(qmf))
    L1=min(L,Lmax)
    WT= pywt.wavedecn(I, qmf, mode='per', level=L1)
    arr, coeff_slices,coeff_shapes = pywt.ravel_coeffs(WT)
    Ind=np.argsort(np.abs(arr))
    WTS=np.zeros_like(arr)
    WTS[Ind[N1*N2-N:N1*N2]]=arr[Ind[N1*N2-N:N1*N2]]
    coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices,coeff_shapes)
    Irec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    p=PSNR(Irec,I)
    return Irec,p

In [ ]:
Irec,p=ApproxOnd2nonlin(im2,'db2',6,5000)
pn.Row(hv.Image(im2,label="Image originale").opts(**options),hv.Image(Irec,label="Image reconstruit par approximation non linéaire de N coefficients").opts(**options))

**Ici, plutôt que de restreindre le nombre de coefficients d'ondelettes par un seuil, nous le définissons directement avec N. Cette approche entraîne également une grande dégradation de la qualité de l'image.**

Créer un Dashboard qui permet d'explorer la fonction précédente.

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3','sym2','sym3']

In [ ]:
class Approx2D(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(5,bounds=(0,7))
    N = param.Integer(2000,bounds=(1,10000))
  #  @param.depends('wave', 'N', 'L')
    def view(self):
        im = imagesRef[self.image]
        Irec,p = ApproxOnd2nonlin(im,self.wave,self.L,self.N)
        return pn.Column(pn.Row(hv.Image(im,label = "Image originale").opts(**options),hv.Image(Irec, label = "Image reconstruit").opts(**options),"PSNR :",p))

In [ ]:
approx2D = Approx2D()
pn.Row(approx2D.param,approx2D.view)

**L'analyse de nos résultats révèle plusieurs observations importantes.** <br>
**- Tout d'abord, nous constatons qu'une augmentation du nombre N de coefficients d'ondelettes conservés se traduit par une amélioration du PSNR et donc une meilleure qualité d'image reconstruite.** <br>
**- Nous notons que les ondelettes de Haar présentent un PSNR inférieur à celui des ondelettes de Daubechies ou de Coifman. Cette différence peut être attribuée à leur nature constante par morceaux, qui les rend moins adaptées aux caractéristiques des images étudiées.** <br> 
**- En outre, il est intéressant de souligner que l'élévation de la profondeur de la transformation en ondelettes L conduit à une augmentation du PSNR, et par conséquent à une meilleure qualité d'image reconstruite.**

Créer un plan d'experiences qui permet d'explorer la fonction ApproxOnd2nonlin

In [ ]:
import itertools
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences = {'Image':imagesRef,'N':np.linspace(1000,50000,30),'wave':wavelist}
dfexp = pd.DataFrame(list(itertools.product(*experiences.values())),columns=experiences.keys())

In [ ]:
print(dfexp)

Créer la fonction qui à une ligne de la base de donnée précédente calcule le PSNR associé.

In [ ]:
def DebruitPSNR(I,qmf,N): #,sigma,T):
    seednoise=np.arange(N)
    N1=len(I.flatten())
    Lmax=pywt.dwt_max_level(len(I),pywt.Wavelet(qmf).dec_len)
    Seuil=10 #T*sigma
    psnr1=np.zeros(N)
    for k in seednoise:
        np.random.seed(seed=seednoise)
        bruit=np.random.normal(0,1,(N1,N1))
        IB=I+sigma*bruit
        Irec=SeuillageOndelette(IB,qmf,Lmax,Seuil)
        psnr1[k]=psnr(I,Irec)
    return np.mean(psnr1)

In [ ]:
def row2PSNR(row):
    Lmax = pywt.dwt_max_level(len(imagesRef[row.Image]),pywt.Wavelet(row.wave).dec_len)
    Irec,p=ApproxOnd2nonlin(imagesRef[row.Image],row.wave,Lmax,int(row.N))
    return {'PSNR':p}
   # p=DebruitPSNR(imagesRef[im],row.wave,row.N) #,row.sigma,row.Th) l'ancien version

Appliquer la fonction sur la base de donnée et ajouter la colonne PSNR à la base de données dfexp 

In [ ]:
result = dfexp.apply(row2PSNR,axis=1)
dfexp[['PSNR']] = pd.DataFrame.from_records(result.values)

In [ ]:
print(dfexp)

Utiliser hvplot pour visualiser la base de données.

In [ ]:
import hvplot.pandas

In [ ]:
from bokeh.models import HoverTool
h = HoverTool()

In [ ]:

dfexp.hvplot('N','PSNR',by='wave',kind='scatter',groupby=['Image']).opts(width=600,tools = [h]).redim.range(PSNR=(22,55))


**Pour l'image Lenna, nous observons que l'ondelette de Coifman 3 (en jaune) fonctionne le mieux, ce qui se traduit par un PSNR plus élevé. Les mêmes résultats sont observés pour l'image Canaletto. Egalement, lorsque le nombre N de coefficients d'ondelettes augmente, la qualité de l'image approximée augmente également, ce qui est cohérent avec nos observations précédentes.**

In [ ]:
print(dfexp)

## Débruitage d'images

Ecrire une fonction qui effectue un seuillage dur en ondelettes et la tester. On pourra utiliser la fonction pywt.ravel_coeffs et on pensera à cliper le résultat entre 0 et 255.

In [ ]:
def SeuillageDurOndelettes(I,qmf,L,Seuil):
    WTB= pywt.wavedecn(I, qmf, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WTB)
    WTS=arr*(np.abs(arr)>Seuil)
    coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices, coeff_shapes)
    Irec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return Irec

Construire un dashboard qui permet d'explorer la fonction SeuillageDurOndelettes.

In [ ]:
class WaveSeuillage(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(10,bounds=(1,1000))
    def view(self):
        im = imagesRef[self.image]
        Irec = SeuillageDurOndelettes(im,self.wave,self.L,self.Seuil)
        p = PSNR(im,Irec)
        return pn.Column(pn.Row(hv.Image(im, label = "Image originale").opts(**options),hv.Image(Irec, label = "Image Reconstruit").opts(**options)),"PSNR",p) #hv.Image(im).opts(**options)*hv.Image(Irec).opts(**options)

In [ ]:
#Affichage du dashboard
Wavethresholding = WaveSeuillage()
pn.Row(Wavethresholding.param,Wavethresholding.view)

**Quand le seuil augumente, l'image recontruite devient plus en plus dégradée. Cela est due au fait qu'on ne garde que les coefficients d'ondelettes plus grands que ce seuil. Nous remarquons que L n'a pas trop d'impact quand la valeur du seuil est petit. Quand on augumente le seuil et L à la fois, l'image est plus lisible que lorsque L est petit. Aussi, la valeur du PSNR est plus petite pour les ondelettes de Haar.**

In [ ]:
#Bruitage de l'image
n1,n2=np.shape(im)
B=np.random.randn(n1,n2)
sigma=10
ib=im+sigma*B
ib=np.clip(ib,0,255)
hv.Image(ib).opts(cmap='gray',width=400,height=400)

Ecrire un dashboard qui permet de visualiser rapidement l'effet d'un débruitage en ondelettes et qui renvoie les images originales, bruitées et débruitées ainsi que les PSNR associés aux images bruitéeset débruitées.

In [ ]:
def Debruit(I,qmf,seednoise,sigma,T, L):
    N1=len(I)
    np.random.seed(seed=seednoise)
    bruit=np.random.normal(0,1,(N1,N1))
    Lmax=pywt.dwt_max_level(len(I),pywt.Wavelet(qmf).dec_len)
    IB=I+sigma*bruit
    Seuil=T*sigma
    Irec=SeuillageDurOndelettes(IB,qmf,L,Seuil)
    p1=PSNR(I,IB)
    p2=PSNR(I,Irec)
    return Irec,IB,p1,p2

In [ ]:
class WaveDebruit(param.Parameterized):
    image = param.ObjectSelector(default="Lenna",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(3,bounds=(1,6))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self):
        im = imagesRef[self.image]
        Irec,IB,p1,p2 = Debruit(im,self.wave,self.seednoise,self.Sigma, self.Seuil, self.L)
        return pn.Column(pn.Row(hv.Image(im, label = "Image originale").opts(**options),hv.Image(IB,label = "Image bruitée").opts(**options),hv.Image(Irec,label = "Image débruitée").opts(**options)),"PSNR entre l'image bruitée et l'image originale: ", p1,"#PSNR entre l'image reconstruite et l'image originale : ", p2)

In [ ]:
wavedebruit = WaveDebruit()
pn.Row(wavedebruit.param,wavedebruit.view)
#il faut verifier le gain en pnsr

**Le PSNR est plus élevée entre l'image reconstruite et l'image originale. Il est de meilleure qualité que celui d'image bruitée. Nous observons les même tendances que la partie précedente, à savoir quand le seuil augumente, la qualité de l'image baisse.**

## Débruitage d'images et translations

Ecrire une fonction qui réalise un débruitage avec une moyenne sur des NbT fois NbT translations et la tester. Vérifier le gain en PNSR.

In [ ]:
def DebruitTranslation(IB,wave,seuil,NbT):
    Lmax=pywt.dwt_max_level(len(IB),pywt.Wavelet(wave).dec_len)
    ISum=0*IB
    P=np.zeros((NbT,NbT))
    rounds = 0
    for k in np.arange(0,NbT):
        for i in np.arange(0,NbT):
            rounds +=1
            IBtemp=np.roll(IB,(k, i),axis = (0,1))
            Irectemp=SeuillageDurOndelettes(IBtemp,wave,Lmax,seuil)
            Irectemp2=np.roll(Irectemp,(-k, -i),axis = (0,1))
            ISum += Irectemp2
            Irec=ISum/rounds
    P=PSNR(IB,Irec)
    return Irec,P

In [ ]:
n1,n2=np.shape(im2)
sigma = 25
br=np.random.randn(n1,n2) #On tire aleatoirement le bruit
IB=im2+sigma*br
Irec, P = DebruitTranslation(IB,'db2',seuil=30,NbT=10)
P2 = PSNR(IB,im2)
P3 = PSNR(Irec,im2)
print(Irec.shape)
pn.Column(pn.Row(hv.Image(IB,label="Image bruitée").opts(**options),
       hv.Image(Irec,label="Image débruitée").opts(**options)),#"PSNR entre l'image bruitée et l'image recontruite: ",P ,
          "PSNR entre l'image originale et l'image bruitée: ", P2,
          "PSNR entre l'image originale et l'image reconstruite: ",P3)

Créer un dasboard pour explorer la fonction précédente. La sortie doit aussi être composée de 3 images et 2 PSNR.

Commenter porquoi 3*sigma est la valeur optimal du seuil. 

**Le produit scalaire $<x+ae, \psi> = <x,\psi> + <ae,\psi>$. Et $<ae,\psi>$ suit une loi normale de paramètres $(0,\sigma^2)$. Donc on sait que le bon seuil est de 3 écart-types i.e. $3 \sigma$**

In [ ]:
class Debruit_translat(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    NbT = param.Integer(2,bounds=(1,8))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self): 
        im = imagesRef[self.image] 
        N1 = len(im)
        np.random.seed(seed=self.seednoise)
        bruit=np.random.normal(0,1,(N1,N1))
        IB = im + self.Sigma*bruit
        Irec, p = DebruitTranslation(IB,self.wave,3*self.Sigma,self.NbT)
        p1=PSNR(IB,im) #PSNR(I,Iref)
        p2=PSNR(Irec,im)
        return pn.Column(pn.Row(hv.Image(im).opts(**options),hv.Image(Irec).opts(**options),hv.Image(IB).opts(**options)),"PSNR entre l'image originale et l'image bruitée",p1,"PSNR entre l'image originale et l'image debruitée",p2)

In [ ]:
debruit_trans = Debruit_translat()
pn.Row(debruit_trans.param,debruit_trans.view)

**Comme nous avons vu en cours, la qualité de reconstruction est meilleure quand on rajoute des translations. Nous le voyons parce que quand le nombre de translations NbT augume, le gain en PSNR augumente aussi.**

## Débruitage d'une image couleur.

Pour effectuer le débruitage d'une image générale, c'est à dire d'une image couleur dont le format n'est pas carré et dont les dimensions ne sont pas des puissnaces de 2 on procède comme suit :

1) On effectue un débruitage séparé sur chacun des canaux.

2) Le format carré n'est pas un vrai problème, il faut juste que les dimensions soit des multiples de puissances de 
2. C'est la puissance de 2 qui définira l'échalle maximale de la décomposition en ondelettes. Il est donc préférable que les dimensions de l'images soient un petit multiple d'une puissance de 2.

3) On étend l'image par symétrie ou périodicité pour qu'elle ait les dimensions souhaitées. A la fin du processus de débruitage on tronque le résultat obtenu à la dimension de l'image originale.

4) Si le niveau de bruit n'est pas connu, il faut l'estimer en utilisant les coefficients d'ondelettes de la plus petite échelle (voir le notebook sur le débruitage de signaux).


# Proposer une fonction qui effectue le débruitage d'une image couleur de dimensions quelconques. 

La fonction peut prendre en entrée un tableau numpy ou une image dans une format d'images classique.
Vous pouvez tester votre programme en bruitant vous même une ou plusieurs images de référence et évaluer le gain en terme de PSNR.

In [ ]:
def symetrize_image(image, desired_shape):
    
    sym_image = np.zeros(desired_shape)
    sym_image[:image.shape[0], :image.shape[1], :] = image
    # Symmetrize along rows
    for i in range(image.shape[0], desired_shape[0]):
        sym_image[i, :, :] = sym_image[2 * image.shape[0] - i - 1, :, :]
    # Symmetrize along columns
    for j in range(image.shape[1], desired_shape[1]):
        sym_image[:, j, :] = sym_image[:, 2 * image.shape[1] - j - 1, :]
    return sym_image


In [ ]:
def PSNR_Couleur (I,Iref):
    n1,n2,n3 = np.shape(I)
    mse = np.mean((Iref-I)**2)
    if mse == 0:
        return 100
    VALMAX = np.max(Iref)
    mse=(1/n1*n2*n3)*np.sum((I-Iref)**2)
    return 20*np.log10(VALMAX**2/np.sqrt(mse))

In [ ]:
def Debruitage_image_couleur(image, qmf, seednoise, sigma, T, L, NbT):
    Seuil=T*sigma
    np.random.seed(seed=seednoise)
    noise = np.random.normal(0, 1, image.shape)
    noisy_image = image + sigma * noise
    
    # Symmetrize image to match desired shape
    desired_shape = (image.shape[0] + image.shape[0] % 2, image.shape[1] + image.shape[1] % 2, 3)
    noisy_image = symetrize_image(noisy_image, desired_shape)
    ISum=np.zeros(np.shape(image))
    Irec=np.zeros(np.shape(image))
    denoised_channels = []
    for channel in range(3):
        channel_image = noisy_image[:,:, channel]
        for i in np.arange(0,NbT):
            IBtemp=np.roll(channel_image,i,axis = 1)
            IBtemp=np.roll(IBtemp,i,axis = 0)
            denoised_channel = SeuillageDurOndelettes(IBtemp,qmf,L,Seuil)
            #denoised_channels.append(denoised_channel)
            denoised_channel2=np.roll(denoised_channel,-i,axis = 1)
            denoised_channel2=np.roll(denoised_channel2,-i,axis = 0)
            ISum[:,:,channel]=ISum[:,:,channel]+denoised_channel2
            Irec[:,:,channel]=ISum[:,:,channel]/(i+1)
    # Reconstruct denoised image
   # Irec = np.stack(denoised_channels, axis=-1)
    
    # Compute PSNR
    psnr1 = PSNR_Couleur(image, noisy_image)
    psnr2 = PSNR_Couleur(image, Irec)
    
    return Irec, noisy_image, psnr1, psnr2

In [ ]:
Ca = chargeData('Cartoon')
Mi = chargeData('Minotaure')
imagesRef2 = {"Cartoon" : Ca ,"Minotaure" : Mi}
pn.Row(hv.Raster(imagesRef2["Cartoon"]).opts(**options2),hv.Raster(imagesRef2["Minotaure"]).opts(**options2))

In [ ]:
#test du fonction
# Paramètres pour le bruitage et le débruitage
image_reference = chargeData("Cartoon")
qmf = 'db1'  # Choix de l'ondelette
seednoise = 42  # Graine pour la génération du bruit
sigma = 20  # Écart-type du bruit gaussien
T = 2  # Facteur de seuil pour le seuillage des coefficients d'ondelettes
L = 3  # Niveau maximal de décomposition en ondelettes
NbT = 5
# Débruitage de l'image bruitée
image_denoised, noisy_image, psnr1, psnr2 = Debruitage_image_couleur(image_reference, qmf, seednoise, sigma, T, L,NbT)

# Affichage des images et des valeurs de PSNR
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(image_reference.astype('uint8'))
plt.title('Image de référence')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(noisy_image.astype('uint8'))
plt.title(f'Image bruitée\nPSNR: {psnr1:.2f} dB')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(image_denoised.astype('uint8'))
plt.title(f'Image débruitée\nPSNR: {psnr2:.2f} dB')
plt.axis('off')

plt.show()

## Pour aller plus loin (à titre informatif et optionnel)

On peut améliorer les méthodes par seuillage dans une base d'ondelettes en effectuant un seuillage par blocs. C'est à dire, ne pas décider de conserver ou pas un coefficients en fonction de sa seule amplitude mais plutôt en fonction de l'énergie d'un voisinage de coefficients. 

Voir : http://www.cnrs.fr/insmi/spip.php?article265

En effet, il est rare qu'un coefficient soit significatif seul au milieu de coefficients nuls. 

La mméthode de sueillage par blocs consiste à choisir une taille de voisinage (par exemple 4*4 coeffients en dimension 2) pour une échelle et une direction donnée et de conserver l'intégralité des coefficients si l'énergie (la somme des carrés des coefficients) est supérieure à un seuil et de les mettre tous à 0 si ce n'est pas le cas. 

Dans ce cas aussi, les translations permettent d'améliorer le rendu visuel en limitant les effets de blocs.

On peut aussi constuire des blocs "3D" en considérant des blocs qui comprennent les coefficients des 3 créneaux de couleurs. L'idée est de corréler le débruitage un peu à travers l'espace et l'espace des couleurs.

Il est possible d'effectuer un débruitage en changeant d'espace colorimétrique en passant du RGB au YUV par exemple.

# Débruiter un minotaure ?

A l'aide de tout ce qui a été fait précédemment, proposer une version débruitée de l'image couleur contenue dans le tableau Mi

In [ ]:
Minotaure=np.clip(Mi,0,255)
hv.RGB(Minotaure.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500)

In [ ]:
def Debruitage_image_couleur_b(image, qmf, sigma, T, L, NbT):
    Seuil=T*sigma
    
    # Symmetrize image to match desired shape
    desired_shape = (image.shape[0] + image.shape[0] % 2, image.shape[1] + image.shape[1] % 2, 3)
    image = symetrize_image(image, desired_shape)
    ISum=np.zeros(np.shape(image))
    Irec=np.zeros(np.shape(image))
    denoised_channels = []
    for channel in range(3):
        channel_image = image[:,:, channel]
        for i in np.arange(0,NbT):
            IBtemp=np.roll(channel_image,i,axis = 1)
            IBtemp=np.roll(IBtemp,i,axis = 0)
            denoised_channel = SeuillageDurOndelettes(IBtemp,qmf,L,Seuil)
            #denoised_channels.append(denoised_channel)
            denoised_channel2=np.roll(denoised_channel,-i,axis = 1)
            denoised_channel2=np.roll(denoised_channel2,-i,axis = 0)
            ISum[:,:,channel]=ISum[:,:,channel]+denoised_channel2
            Irec[:,:,channel]=ISum[:,:,channel]/(i+1)
    # Reconstruct denoised image
   # Irec = np.stack(denoised_channels, axis=-1)
    
    psnr2 = PSNR_Couleur(image, Irec)
    
    return Irec, psnr2

In [ ]:
class Debruit_translat_Minotaure(param.Parameterized):
    wave = param.ObjectSelector(default="db2",objects=wavelist)
    NbT = param.Integer(4,bounds=(1,8))
    Sigma = param.Number(10,bounds=(1,100))
    L = param.Integer(7,bounds=(0,7))
    T=param.Number(3,bounds=(0,8))
    def view(self):
        #S = Mi
        #Minotaure=np.clip(Mi,0,255)

        #Débruitage :
        Irec, p = Debruitage_image_couleur_b(Mi,self.wave,self.L,self.T,self.Sigma,self.NbT)
        #MinotaureRec=np.clip(Irec,0,255)

        Ibr = hv.RGB(Minotaure.astype('uint8'),label="Image bruitée originale").opts(xlabel=None,ylabel=None,width=350,height=350)
        Ideb = hv.RGB(Irec.astype('uint8'),label = "Image débruitée").opts(xlabel=None,ylabel=None,width=350,height=350)

        return pn.Column(pn.Row(Ibr,Ideb),"PSNR entre l'image bruitée originale et l'image debruitée", p)

In [ ]:
DebruitTranslatMinotaure = Debruit_translat_Minotaure()
pn.Row(DebruitTranslatMinotaure.param,DebruitTranslatMinotaure.view)

Rédiger également une fonction prenant en entrée un nom de fichier 
permettant de calculer le PSNR de votre proposition d'image débruitée avec l'image en question.
On calcule le PSNR entre deux images couleurs en calculant la somme des erreurs quadratiques sur les 3 canaux.

Attention, l'image a 3 canaux de couleur, n'est pas carrée et les dimensions ne sont pas des puissances de 2.

### Plan d'expériences pour évaluer l'impact des translations  

Créer un plan d'expériences pour explorer les performances de l'invariance par translation pour le débruitage. 

In [ ]:
experiences_DebruitTrans = {'Image':imagesRef,'NbT':np.arange(1,5),'wave':wavelist,'Sigma':np.linspace(10,30,2)}
dfexp_DebruitTrans = pd.DataFrame(list(itertools.product(*experiences_DebruitTrans.values())),columns=experiences_DebruitTrans.keys())

In [ ]:
print(dfexp_DebruitTrans)

Ecrire une fonction qui calcule le PSNR moyen sur n réalisations de bruit du débruitage d'une image avec NbT*NbT translations (qui utilise par exemple la fonction DebruitTranslation)

In [ ]:
def Debruit_Translat_PSNRMoyen(I,wave,sigma,NbT,n):
    P2=0
  #  I=np.clip(I,0,255)
    n1,n2 = I.shape
    for i in np.arange(0,n):
        np.random.seed(seed=20)
        noise = np.random.normal(0, 1, (n1,n2))
        #noise = np.random.randn(n1,n2)
        IB= I +sigma*noise
       # IB=np.clip(IB,0,255)
        Irec,P = DebruitTranslation(IB,wave,3*sigma,NbT)
        #Irec=np.clip(Irec,0,255)
        P2 += PSNR(Irec,I)
    return P2/n

Ecrire la fonction qui à une ligne de la base de données précédente calcule le PSNR moyen sur 4 réalisations du bruit. Puis l'appliquer à la base de données et ajouter la colonne des PSNR calculés à la base de données.

In [ ]:
def row2DebruitTrans(row):
    Pmoy=Debruit_Translat_PSNRMoyen(imagesRef[row.Image],row.wave,row.Sigma,row.NbT,4)
    return {'PSNR':Pmoy}

In [ ]:
result2 = dfexp_DebruitTrans.apply(row2DebruitTrans,axis=1)
dfexp_DebruitTrans[['PSNR']] = pd.DataFrame.from_records(result2.values)

In [ ]:
print(dfexp_DebruitTrans)

Utiliser hvplot pour visualiser les résulatst contenus dans la base de données.

In [ ]:
import hvplot

In [ ]:
dfexp_DebruitTrans.hvplot('NbT','PSNR',by='wave',kind='scatter',groupby=['Sigma','Image']).opts(width=600,tools = [h]).redim.range(PSNR=(0,55),NbT=(0.5,4.5))

# Quantification et Entropie de Shannon

In [ ]:
def ShannonEntropy(x):
    value,counts = np.unique(x, return_counts=True)
    Proba=counts/len(x)
    Ent=-np.sum(np.log2(Proba)*Proba)
    return Ent

In [ ]:
x=np.array([13,13,2,7,13,7,1,13])
print(ShannonEntropy(x))

In [ ]:
y=np.array([-2,-3,1,0,1,0,-2,-3])
print(ShannonEntropy(y))

Ecrire une fonction qui effectue la quantification de la transformée en ondelettes avec un pas "Pas". On pourra à nouveau utiliser la commande pywt.ravel_coeffs. La fonction doit renvoyer l'image calculée par quantification, le PSNR associé ainsi que le nombre d'octets estimé par la valeur de l'entropie a priori nécessaire pour coder une telle image. On considérera qu'on code séparément les coefficients d'échelle et les coefficients d'ondelettes. Tester la fonction.

In [ ]:
#def QuantificationOndelettes(I,qmf,Pas):
    

Créer le dashboard asscoié à la fonction précédente. 
Le dashboard doit renvoyer l'image quantifiée, le PSNR de l'image ainsi que le facteur de compression théorique associé. 

In [ ]:
class WaveQuant(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    QS = param.Number(30,bounds=(10,300))
    ...

Créer un plan d'expériences pour comparer les différentes ondelettes pour la quantification... et poursuivre jusqu'à obtenir un affichage de la base de données ainsi créée avec hvplot.

In [ ]:
experiences_quant = {'QS':np.linspace(30,200,10),'wave':wavelist}
...

In [ ]:
def row2DistorsionRate(row):
    

## Pour aller plus loin (à titre informatif et optionnel)

Nous proposons ici d'effectuer la compression sur les 3 canau RGB. Or l'oeil humain est plus sensible à la luminance qu'aux composantes purement chromatiques. C'est pourquoi, la plupart des algorithmes de compressions sont effectué dans un espace colorimétrique YUV où Y est la luminance. On alloue alors plus d'information au canal Y et on comprime plus drastiquement les deux autres canaux. Une méthode standart consiste par exemple à sous-échantionner d'un facteur 2 les deux composantes U et V avant de les comprimer. 

https://fr.wikipedia.org/wiki/Sous-échantillonnage_de_la_chrominance

On obtient alors des images de chrominances moins résolues et donc moins lourdes mais le rendu final reste correct car l'oeil humain est nettement plus sensible à la luminance. 

In [ ]:
import numpy as np
import pywt

def symetrize_image(image, desired_shape):
    """
    Extend image by symmetrization to match desired_shape.
    """
    sym_image = np.zeros(desired_shape)
    sym_image[:image.shape[0], :image.shape[1], :] = image
    # Symmetrize along rows
    for i in range(image.shape[0], desired_shape[0]):
        sym_image[i, :, :] = sym_image[2 * image.shape[0] - i - 1, :, :]
    # Symmetrize along columns
    for j in range(image.shape[1], desired_shape[1]):
        sym_image[:, j, :] = sym_image[:, 2 * image.shape[1] - j - 1, :]
    return sym_image

def Debruitage_image_couleur(image, qmf, seednoise, sigma, T, L):
    """
    Perform denoising on a color image using wavelet thresholding.
    
    Arguments:
    image : numpy array
        Input color image.
    qmf : str
        Name of the orthogonal wavelet to use.
    seednoise : int
        Seed for the random number generator for noise generation.
    sigma : float
        Standard deviation of the Gaussian noise.
    T : float
        Threshold factor for wavelet coefficient thresholding.
    L : int
        Maximum level of wavelet decomposition.
    
    Returns:
    Irec : numpy array
        Denoised image.
    IB : numpy array
        Noisy image.
    psnr1 : float
        Peak Signal-to-Noise Ratio (PSNR) between the noisy and original image.
    psnr2 : float
        Peak Signal-to-Noise Ratio (PSNR) between the denoised and original image.
    """
    N1, N2, _ = image.shape
    np.random.seed(seed=seednoise)
    noise = np.random.normal(0, 1, (N1, N2, 3))
    noisy_image = image + sigma * noise
    
    # Symmetrize image to match desired shape
    desired_shape = (N1 + N1 % 2, N2 + N2 % 2, 3)
    noisy_image = symetrize_image(noisy_image, desired_shape)
    
    # Wavelet denoising separately on each channel
    denoised_channels = []
    for channel in range(3):
        channel_image = noisy_image[:, :, channel]
        coeffs = pywt.wavedec2(channel_image, qmf, mode='symmetric', level=L)
        denoised_coeffs = list(map(lambda x: pywt.threshold(x, T * sigma, mode='soft'), coeffs))
        denoised_channel = pywt.waverec2(denoised_coeffs, qmf, mode='symmetric')[:N1, :N2]
        denoised_channels.append(denoised_channel)
    
    # Reconstruct denoised image
    Irec = np.stack(denoised_channels, axis=-1)
    
    # Compute PSNR
    psnr1 = PSNR(image, noisy_image)
    psnr2 = PSNR(image, Irec)
    
    return Irec, noisy_image, psnr1, psnr2

#def PSNR(original, noisy):
    """
    Compute Peak Signal-to-Noise Ratio (PSNR) between original and noisy images.
    """
   # mse = np.mean((original - noisy) ** 2)
   # if mse == 0:
   #     return float('inf')
   # max_pixel = 255.0
   # psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
   # return psnr
